# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Nasir  ->  F. Nasir  |  ['F. Nasir']
P. Gaikwad  ->  P. Gaikwad  |  ['P. Gaikwad']
A. Gould  ->  A. Gould  |  ['A. Gould']
J. Liu  ->  J. Liu  |  ['J. Liu']
C. Fendt  ->  C. Fendt  |  ['C. Fendt']
Arxiv has 47 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.05794


extracting tarball to tmp_2404.05794...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

extracting tarball to tmp_2404.05975... done.
Retrieving document from  https://arxiv.org/e-print/2404.06140


extracting tarball to tmp_2404.06140...

 done.


C. Fendt  ->  C. Fendt  |  ['C. Fendt']


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.06140-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.06140) | **Thin Accretion disks in GR-MHD simulations**  |
|| I. K. Dihingia, <mark>C. Fendt</mark> |
|*Appeared on*| *2024-04-10*|
|*Comments*| *37 pages, 15 figures, Invited chapter for the edited book New Frontiers in GRMHD Simulations (Eds. C. Bambi, Y. Mizuno, S. Shashank and F. Yuan, Springer Singapore, expected in 2024)*|
|**Abstract**| We review some recent results of general relativistic magnetohydrodynamic (GR-MHD) simulations considering the evolution of geometrically thin disks around a central black hole. Thin disk GR-MHD simulations complement the widely used MAD (Magnetically Arrested Disk) or SANE (Standard And Normal Evolution) approaches of evolving from an initial disk torus. In particular, we discuss the dynamical evolution of the disk, its role in the formation of disk winds or jets, the impact of disk resistivity, and its potential role in generating magnetic flux by an internal disk dynamo. The main characteristics of a thin disk in our approach are the Keplerian rotation of the disk material, which allows to launch disk outflows by the Blandford-Payne magneto-centrifugal effect, in addition to the Blandford-Znajek-driven spine jet from the black hole ergosphere. Thus, for this approach, we neglect disk thermodynamics and radiative effects, concentrating predominantly on the dynamical evolution of the system. Resistive MHD further allows the investigation of physical reconnection and also dynamo action. Magnetic reconnection may generate magnetic islands of plasmoids that are ejected from the disk along with the outflow. We also discussed potential applications of thin disk in explaining the decaying phase of an outburst in black hole X-ray binaries (BH-XRBs). Post-processing of radiation using the simulated dynamical data allows to derive spectra or fluxes, e.g., in the X-ray band, and to derive potential variability characteristics. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.05975-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.05975) | **On Variation of Light Curves and Broad Emission Lines for Periodic QSOs  from co-rotating Supermassive binary black holes in elliptical orbits**  |
|| J. Ge, Y. Lu, C. Yan, <mark>J. Liu</mark> |
|*Appeared on*| *2024-04-10*|
|*Comments*| *13 pages, 9 figures, accepted for publication in A&A*|
|**Abstract**| Context. Periodic QSOs are considered as candidates of supermassive binary black hole (BBH) systems in galactic centers. Further confirmation of these candidates may require different lines of observational evidences. Aims. Assuming the Doopler boosting scenario, in this paper we investigate the (coherent) variations of both broad emission lines (BELs) and continuum light curves for active BBH systems surrounding by a circumbinary broad line region (cBLR) and focus on their dependence on the eccentric orbital configuration. Methods. We calculate the variation of continuum light and the Doppler enhanced/weakened photoionization of each BLR cloud according to the motion of BBHs in elliptical orbits, and finally obtain the coherent variation of the continuum and BELs. Results. We find that both the amplitude and variation pattern of the continuum light curves and the evolution of the BEL profiles sensitively depend on the eccentric orbital configuration of BBH systems. If only the secondary BH is active, both the variation amplitudes of continuum light curves and BELs increase with increasing BBH inclination angles and orbital eccentricities, but decrease with increasing BBH mass ratio. If both BHs are active, the asymmetry in the ionization of BLR clouds at different areas caused by the Doppler boosting effect of the secondary BH is weakened due to that of the primary BH at the opposite direction, which leads to systematically smaller variation amplitudes of both continuum light curves and BELs than the cases with only secondary BH activated. Conclusions. The coherent variations of the BEL profiles with the continuum light for those periodic QSOs provide an important way to confirm the existence of BBHs in their center. Future joint analysis of the light curves and multi-epoch observed BEL profiles for periodic QSOs may lead to the identification of a number of BBH systems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.05912-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.05912) | **OGLE-2018-BLG-0971, MOA-2023-BLG-065, and OGLE-2023-BLG-0136:  Microlensing events with prominent orbital effects**  |
|| C. Han, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2024-04-10*|
|*Comments*| *11 pages, 13 figures, 6 tables*|
|**Abstract**| We undertake a project to reexamine microlensing data gathered from high-cadence surveys. The aim of the project is to reinvestigate lensing events with light curves exhibiting intricate anomaly features associated with caustics, yet lacking prior proposed models to explain these features. Through detailed reanalyses considering higher-order effects, we identify that accounting for orbital motions of lenses is vital in accurately explaining the anomaly features observed in the light curves of the lensing events OGLE-2018-BLG-0971, MOA-2023-BLG-065, and OGLE-2023-BLG-0136. We estimate the masses and distances to the lenses by conducting Bayesian analyses using the lensing parameters of the newly found lensing solutions. From these analyses, we identify that the lenses of the events OGLE-2018-BLG-0971 and MOA-2023-BLG-065 are binaries composed of M dwarfs, while the lens of OGLE-2023-BLG-0136 is likely to be a binary composed of an early K-dwarf primary and a late M-dwarf companion. For all lensing events, the probability of the lens residing in the bulge is considerably higher than that of it being located in the disk. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.05794-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.05794) | **Deep Learning the Intergalactic Medium using Lyman-alpha Forest at $ 4  \leq z \leq 5$**  |
|| <mark>F. Nasir</mark>, et al. -- incl., <mark>P. Gaikwad</mark> |
|*Appeared on*| *2024-04-10*|
|*Comments*| *17 pages, Submitted to MNRAS*|
|**Abstract**| Unveiling the thermal history of the intergalactic medium (IGM) at $4 \leq z \leq 5$ holds the potential to reveal early onset HeII reionization or lingering thermal fluctuations from HI reionization. We set out to reconstruct the IGM gas properties along simulated Lyman-alpha forest data on pixel-by-pixel basis, employing deep Bayesian neural networks. Our approach leverages the Sherwood-Relics simulation suite, consisting of diverse thermal histories, to generate mock spectra. Our convolutional and residual networks with likelihood metric predicts the Ly$\alpha$ optical depth-weighted density or temperature for each pixel in the Ly$\alpha$ forest skewer. We find that our network can successfully reproduce IGM conditions with high fidelity across range of instrumental signal-to-noise. These predictions are subsequently translated into the temperature-density plane, facilitating the derivation of reliable constraints on thermal parameters. This allows us to estimate temperature at mean cosmic density, $T_{\rm 0}$ with one sigma confidence $\delta T_{\rm 0} \sim 1000{\rm K}$ using only one $20$Mpc/h sightline ($\Delta z\simeq 0.04$) with a typical reionization history. Existing studies utilize redshift pathlength comparable to $\Delta z\simeq 4$ for similar constraints. We can also provide more stringent constraints on the slope ($1\sigma$ confidence interval $\delta {\rm \gamma} \lesssim 0.1$) of the IGM temperature-density relation as compared to other traditional approaches. We test the reconstruction on a single high signal-to-noise observed spectrum ($20$ Mpc/h segment), and recover thermal parameters consistent with current measurements. This machine learning approach has the potential to provide accurate yet robust measurements of IGM thermal history at the redshifts in question. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.06140.md
    + _build/html/tmp_2404.06140/./wind-prop.png
    + _build/html/tmp_2404.06140/./ideal-hd.png
    + _build/html/tmp_2404.06140/./apjabe93bf16_hr_cr_1.jpeg


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ic}[1]{\textcolor{red}{ #1}}$
$\newcommand{\cf}{\textcolor{purple}}$
$\newcommand{\enquote}[1]{"#1"}$</div>



<div id="title">

# Thin Accretion disks in GR-MHD simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.06140-b31b1b.svg)](https://arxiv.org/abs/2404.06140)<mark>Appeared on: 2024-04-10</mark> -  _37 pages, 15 figures, Invited chapter for the edited book New Frontiers in GRMHD Simulations (Eds. C. Bambi, Y. Mizuno, S. Shashank and F. Yuan, Springer Singapore, expected in 2024)_

</div>
<div id="authors">

I. K. Dihingia, <mark>C. Fendt</mark>

</div>
<div id="abstract">

**Abstract:** We review some recent results of general relativistic magnetohydrodynamic (GR-MHD) simulations considering the evolution of geometrically thin disks around a central black hole.Thin disk GR-MHD simulations complement the widely used MAD (Magnetically Arrested Disk) or SANE (Standard And Normal Evolution) approaches of evolving from an initial disk torus.In particular, we discuss the dynamical evolution of the disk, its role in the formation of disk winds or jets, the impact of disk resistivity, and its potential role in generating magnetic flux by an internal disk dynamo.The main characteristics of a thin disk in our approach are the Keplerian rotation of the disk material, which allows to launch disk outflows by the Blandford-Payne magneto-centrifugal effect, in addition to the Blandford-Znajek-driven spine jet from the black hole ergosphere. Thus, for this approach, we neglect disk thermodynamics and radiative effects, concentrating predominantly on the dynamical evolution of the system.Resistive MHD further allows the investigation of physical reconnection and also dynamo action.Magnetic reconnection may generate magnetic islands of plasmoids that are ejected from the disk along with the outflow.We also discussed potential applications of thin disk in explaining the decaying phase of an outburst in black hole X-ray binaries (BH-XRBs). Post-processing of radiation using the simulated dynamical data allows to derive spectra or fluxes, e.g., in the X-ray band,and to derive potential variability characteristics.

</div>

<div id="div_fig1">

<img src="tmp_2404.06140/./wind-prop.png" alt="Fig6" width="100%"/>

**Figure 6. -** Distribution of (a) poloidal Alfvénic Mach number ($M_{\rm A,P}$), (b) $B_{\rm tor}/B_{\rm p}$, and (c) plasma-$\beta$ for a magnetised thin accretion disk evolution from \cite{Dihingia-etal2021}. In panel (a), the solid black and blue lines correspond to contours of magnetisation $\sigma=1$ and $M_{\rm A,P}=1$, respectively. In panel (b),  the solid black, grey, and blue lines correspond to contours of $B_{\rm tor}/B_{\rm p} = 1, 5$, and $M_{\rm A,P} = 1$, respectively. In panel (c), the black and red lines correspond to the contours $\beta=1$, and $M_{\rm A,P} = 1$, respectively. (*fig-wind-prop*)

</div>
<div id="div_fig2">

<img src="tmp_2404.06140/./ideal-hd.png" alt="Fig4" width="100%"/>

**Figure 4. -** Hydrodynamic evolution of NTD setup from \cite{Dihingia-etal2021}. Panel (a) and (b) show density distribution at time $t=0$ and $t=10000 t_g$, respectively. (*fig-hyd*)

</div>
<div id="div_fig3">

<img src="tmp_2404.06140/./apjabe93bf16_hr_cr_1.jpeg" alt="Fig15" width="100%"/>

**Figure 15. -** Evolution magnetic field strength during the dynamo process.
    Shown is the Poynting flux for the funnel BZ jet, and the disk wind (left).
    The flux is integrated along a sphere with radius $r=35 R_{\rm g}$, while the BZ considered
    for the segment $0\deg <\theta < 25\deg$ and the disk wind for the segment
    $25\deg <\theta < 65\deg$. The values are averaged over the corresponding segments at the
    lower hemisphere.
    The corresponding evolution of the absolute dynamo parameter $\xi$ is shown at the bottom,
    averaged in space for the whole disk, and for the inner disk area, $r < 20 R_{\rm g}$.
    Panels taken from \cite{VourellisFendt2021}. (*fig-evolv-poynting*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.06140"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

400  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
